In [76]:
import pandas as pd
import numpy as np
import matplotlib as plt
import xarray as xr
from datetime import datetime

In [3]:
CO2_mix_S1=xr.open_dataset('CO2_per_mix_S1.nc')
CO2_mix_S2=xr.open_dataset('CO2_per_mix_S2.nc')
display(CO2_mix_S1)

<xarray.Dataset>
Dimensions:                        (Date: 365, year: 31, Hour: 24, technology: 9)
Coordinates:
  * Date                           (Date) int32 1 2 3 4 5 ... 362 363 364 365
  * year                           (year) float64 2.02e+03 ... 2.05e+03
  * Hour                           (Hour) int32 0 1 2 3 4 5 ... 19 20 21 22 23
  * technology                     (technology) object 'Autre renouvelables' ...
Data variables:
    __xarray_dataarray_variable__  (Date, year, Hour, technology) float64 ...

In [4]:
year_val=CO2_mix_S1.coords['year'].values
date_val=CO2_mix_S1.coords['Date'].values
hour_val=CO2_mix_S1.coords['Hour'].values

In [7]:
sum_CO2_S1=CO2_mix_S1.sum(dim="technology")
sum_CO2_S2=CO2_mix_S2.sum(dim="technology")

stored_CO2_arrays= (sum_CO2_S1, sum_CO2_S2)

In [40]:
scenario_dic ={}
scenario_dic['S1']={}
scenario_dic['S2']={}

for key,value in enumerate(['S1', 'S2']):
    for yeari in year_val:
        d=pd.DataFrame()
        print(yeari)
        #Date_amounts = []
        for Datei in date_val:
            Hour_amounts = []
            for houri in hour_val:
                    test04=stored_CO2_arrays[key].sel(
                        year=yeari,
                        Date=Datei,
                        Hour=houri,
                    ).to_array().values[0]
                    Hour_amounts.append(test04)
            d2=pd.DataFrame(Hour_amounts)
            d=pd.concat([d,d2])
        scenario_dic[value][yeari]={}
        scenario_dic[value][yeari][0]=d


2020.0
2021.0
2022.0
2023.0
2024.0
2025.0
2026.0
2027.0
2028.0
2029.0
2030.0
2031.0
2032.0
2033.0
2034.0
2035.0
2036.0
2037.0
2038.0
2039.0
2040.0
2041.0
2042.0
2043.0
2044.0
2045.0
2046.0
2047.0
2048.0
2049.0
2050.0
2020.0
2021.0
2022.0
2023.0
2024.0
2025.0
2026.0
2027.0
2028.0
2029.0
2030.0
2031.0
2032.0
2033.0
2034.0
2035.0
2036.0
2037.0
2038.0
2039.0
2040.0
2041.0
2042.0
2043.0
2044.0
2045.0
2046.0
2047.0
2048.0
2049.0
2050.0


In [58]:
for scenario in scenario_dic.keys():
    #print(scenario)
    for yeari in year_val:
        scenario_dic[scenario][yeari][0].reset_index(inplace=True, drop=True)
        #if yeari==2045:
        # if yeari%2==0:
        #     print("EVEN")
        # else:
        #     print("ODD")
            #display(scenario_dic[scenario][yeari])
        if yeari%2==0:
            timestep = pd.Series(pd.date_range(start=f'{int(yeari)}-01-01', end=f'{int(yeari)}-12-31 23:00:00', freq='H'))
            dates = pd.date_range(start=f'{int(yeari)}-01-01', end=f'{int(yeari)}-12-31')
            num_repeats = 24

            scenario_dic[scenario][yeari][0]['Timestamp']=timestep
            scenario_dic[scenario][yeari][0]['Date']= pd.DataFrame(dates.repeat(num_repeats),columns=['date'])
            scenario_dic[scenario][yeari][0]['Hour'] = pd.to_datetime(scenario_dic[scenario][yeari][0]['Timestamp']).dt.strftime('%H:%M:%S')
            scenario_dic[scenario][yeari][0].rename(columns={0:"CO2"}, inplace=True)
            
        else:
            timestep = pd.Series(pd.date_range(start=f'{int(yeari)}-01-01', end=f'{int(yeari)}-12-31 23:00:00', freq='H'))
            dates = pd.date_range(start=f'{int(yeari)}-01-01', end=f'{int(yeari)}-12-31')
            num_repeats = 24

            scenario_dic[scenario][yeari][0]['Timestamp']=timestep
            scenario_dic[scenario][yeari][0]['Date']= pd.DataFrame(dates.repeat(num_repeats),columns=['date'])
            scenario_dic[scenario][yeari][0]['Hour'] = pd.to_datetime(scenario_dic[scenario][yeari][0]['Timestamp']).dt.strftime('%H:%M:%S')
            scenario_dic[scenario][yeari][0].rename(columns={0:"CO2"}, inplace=True)
        if yeari==2028:
            if scenario=="S1":
                display(scenario_dic[scenario][yeari][0])#.to_string())
            

,CO2,Timestamp,Date,Hour
0,0.058565,2028-01-01 00:00:00,2028-01-01,00:00:00
1,0.055515,2028-01-01 01:00:00,2028-01-01,01:00:00
2,0.050192,2028-01-01 02:00:00,2028-01-01,02:00:00
3,0.040667,2028-01-01 03:00:00,2028-01-01,03:00:00
4,0.036473,2028-01-01 04:00:00,2028-01-01,04:00:00
...,...,...,...,...
8755,0.054683,2028-12-30 19:00:00,2028-12-30,19:00:00
8756,0.054261,2028-12-30 20:00:00,2028-12-30,20:00:00
8757,0.053164,2028-12-30 21:00:00,2028-12-30,21:00:00
8758,0.053033,2028-12-30 22:00:00,2028-12-30,22:00:00


In [59]:
for scenario in scenario_dic.keys():
    #print(scenario)
    for yeari in year_val:
        scenario_dic[scenario][yeari][1]=scenario_dic[scenario][yeari][0].copy().groupby('Date').min(numeric_only=True)
        if yeari==2025:
            display( scenario_dic[scenario][yeari][1])


#opt2040_base['CO2']=test2040_base.groupby("Date").min(numeric_only=True)

,CO2
Date,
2025-01-01,0.038456
2025-01-02,0.054558
2025-01-03,0.043494
2025-01-04,0.049802
2025-01-05,0.063358
...,...
2025-12-27,0.032844
2025-12-28,0.033571
2025-12-29,0.034024


,CO2
Date,
2025-01-01,0.038207
2025-01-02,0.054295
2025-01-03,0.043235
2025-01-04,0.049539
2025-01-05,0.063083
...,...
2025-12-27,0.032599
2025-12-28,0.033330
2025-12-29,0.033779


In [84]:
# Assuming you already have your DataFrame
# Let's call it 'df'

# Convert the index to a datetime object (if not already)
#test1=df01.set_index("Timestamp")

for scenario in scenario_dic.keys():
    #print(scenario)
    for yeari in year_val:
        #scenario_dic[scenario][yeari][0]['Timestamp'].to_timestamp()=pd.to_datetime(scenario_dic[scenario][yeari][0]['Timestamp'].to_timestamp())
        grouped=scenario_dic[scenario][yeari][0].groupby(pd.to_datetime(scenario_dic[scenario][yeari][0]['Timestamp']).dt.date)
       # if yeari==2025:
        #    for date , group in grouped:
         #       print(date)
          #      print(group)
                #print
           #     if date==datetime.strptime("2025-05-02", '%Y-%m-%d'):
                    #print(date)
                    #display(grouped)
        dates=[]
        lowest_hour=[]
        for date, group in grouped:
             min_row = group.loc[group['CO2'].idxmin()]
             dates.append(date)
             lowest_hour.append(min_row['Hour'])
        scenario_dic[scenario][yeari][2] = pd.DataFrame({'Date': dates, 'Lowest_CO2_Hour': lowest_hour})
display(scenario_dic['S1'][2035][2])
        #f yeari==2025:
         #   pd.to_datetime(scenario_dic[scenario][yeari][0]['Timestamp']).dt.date
         #   display(times.dt.date)
        #     display(scenario_dic[scenario][yeari][2])





# test1.index = pd.to_datetime(test1.index)

# # Group the DataFrame by the date
# grouped = test1.groupby(test1.index.date)

# # Initialize empty lists to store the result
# dates = []
# lowest_hour = []

# # Iterate through the groups
# for date, group in grouped:
#     # Find the row with the lowest CO2 value for each group
#     min_row = group.loc[group['CO2'].idxmin()]
#     dates.append(date)
#     lowest_hour.append(min_row['Hour'])

# # Create a new DataFrame with the results
# result_df = pd.DataFrame({'Date': dates, 'Lowest_CO2_Hour': lowest_hour})

# # Display the result
# print(result_df)


,Date,Lowest_CO2_Hour
0,2035-01-01,04:00:00
1,2035-01-02,04:00:00
2,2035-01-03,05:00:00
3,2035-01-04,05:00:00
4,2035-01-05,05:00:00
...,...,...
360,2035-12-27,23:00:00
361,2035-12-28,14:00:00
362,2035-12-29,00:00:00
363,2035-12-30,04:00:00
